# Create reports and extract ROI activity for source-level PSDs
For STC power spectra created on 1-second epochs. Based on code originally tested in Explore_STCMethods notebook, and overall structure, including subject groups, are the same as in F2F_PSD_CreateReports. The subsequent ROI analysis is based on the Explore_ROIAnalysis notebook.

From 2021-04-15, using "reduced" adult spectral bands.

On 2021-12-14, new versions of the average ROI tables were made for which the log unit values were converted back to linear units. (Note that the sum of ROI values across bands is always less than 100.)  

On 2022-01-25, transverse temporal and banks of superior temporal sulcus were added to the ROI list.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1">Preliminaries</a></span></li><li><span><a href="#Generate-PSD-STCs" data-toc-modified-id="Generate-PSD-STCs-2">Generate PSD STCs</a></span></li><li><span><a href="#Create-individual-and-group-reports" data-toc-modified-id="Create-individual-and-group-reports-3">Create individual and group reports</a></span></li><li><span><a href="#Group-reports-for-talk-only-and-ignore-only-maps" data-toc-modified-id="Group-reports-for-talk-only-and-ignore-only-maps-4">Group reports for talk-only and ignore-only maps</a></span></li><li><span><a href="#Extract-label-PSDs-for-ROI-analysis---surface-source-space" data-toc-modified-id="Extract-label-PSDs-for-ROI-analysis---surface-source-space-5">Extract label PSDs for ROI analysis - surface source space</a></span></li><li><span><a href="#Repeat-label-extraction-for-all-the-rest-of-the-labels" data-toc-modified-id="Repeat-label-extraction-for-all-the-rest-of-the-labels-6">Repeat label extraction for all the rest of the labels</a></span></li></ul></div>

## Preliminaries

In [1]:
%%capture

import os
import pandas as pd
import importlib

import mne
import mnefun

from f2f_score import score
import f2f_analysis_source
from f2f_analysis_source import create_srcpsd_summary, create_srcpsds

In [2]:
# Define subject groupings #
# Missing now includes f2f_033 and 041; _026 added as of 2021-Jan
all_subjects = ('f2f_009', 'f2f_010', 'f2f_011', 'f2f_012', 'f2f_013', 'f2f_014',
                'f2f_015', 'f2f_018', 'f2f_021', 'f2f_022', 'f2f_024', 'f2f_025',
                'f2f_026', 'f2f_027', 'f2f_028', 'f2f_029', 'f2f_030', 'f2f_034',
                'f2f_036', 'f2f_037', 'f2f_038', 'f2f_102', 'f2f_103', 'f2f_104',
                'f2f_106', 'f2f_107', 'f2f_108', 'f2f_111', 'f2f_112', 'f2f_114',
                'f2f_115', 'f2f_116', 'f2f_117', 'f2f_118', 'f2f_119')   # N=35
                
subjects_SES_high = ('f2f_034', 'f2f_011', 'f2f_022', 'f2f_024', 'f2f_009',
                     'f2f_025', 'f2f_037', 'f2f_103', 'f2f_038', 'f2f_028', 'f2f_115',
                     'f2f_012', 'f2f_036', 'f2f_104', 'f2f_106', 'f2f_112', 'f2f_114',)  # N=17
subjects_SES_low = ('f2f_010', 'f2f_118', 'f2f_116', 'f2f_015', 'f2f_117', 'f2f_107',
                    'f2f_013', 'f2f_119', 'f2f_111', 'f2f_030', 'f2f_018', 'f2f_102', 
                    'f2f_108', 'f2f_026', 'f2f_027', 'f2f_014', 'f2f_021', 'f2f_029')  # N=18
# High: 034, 011, 022, 033, 024, 009, 025, 037, 103, 038, 028, 115, 012, 036, 104,
# 106, 112, 114
# Low: 010, 118, 116, 015, 117, 107, 013, 119, 111, 030, 018, 102, 108, 041, 027,
# 014, 021, 029
# Bad (as of Sep 7 2020): 033 (bad data), 041 (bad HPI)

subjects_vocab24_high = ('f2f_036', 'f2f_106', 'f2f_102', 'f2f_021', 'f2f_018', 'f2f_026',
                         'f2f_011', 'f2f_037', 'f2f_118', 'f2f_111', 'f2f_119',
                         'f2f_108', 'f2f_114', 'f2f_030', 'f2f_034', 'f2f_038')  # N=16
subjects_vocab24_low = ('f2f_015', 'f2f_022', 'f2f_107', 'f2f_010', 'f2f_024', 'f2f_104', 
                        'f2f_025', 'f2f_117', 'f2f_028','f2f_116', 'f2f_012', 'f2f_013',
                        'f2f_027', 'f2f_103', 'f2f_112', 'f2f_029', 'f2f_009')   # N=17
# High: 036, 106, 102, 021, 041, 011, 037, 118, 111, 119, 108, 114, 030, 033, 034, 018, 038
# Low: 015, 022, 107, 010, 024, 104, 025, 117, 028, 116, 012, 013, 027, 103, 112, 029, 009

subjects_sntc24_high = ('f2f_106', 'f2f_037', 'f2f_034', 'f2f_103', 'f2f_102', 'f2f_026',
                        'f2f_011', 'f2f_118', 'f2f_119', 'f2f_013', 'f2f_030', 'f2f_018',
                        'f2f_012', 'f2f_108', 'f2f_111', 'f2f_115', 'f2f_114')  # N=17
subjects_sntc24_low = ('f2f_015', 'f2f_022', 'f2f_107', 'f2f_010', 'f2f_104', 'f2f_027',
                       'f2f_112', 'f2f_028', 'f2f_116', 'f2f_021', 'f2f_025','f2f_117',
                       'f2f_029', 'f2f_009', 'f2f_014', 'f2f_036', 'f2f_024')   # N=17
# High: 106, 037, 034, 103, 102, 011, 118, 119, 013, 012, 030, 041, 108, 111, 115,
# 033, 018, 114
# Low: 015, 022, 107, 010, 024, 104, 117, 027, 112, 028, 116, 021, 025, 029,
# 009, 014, 036

subjects_sntc30_high = ('f2f_102', 'f2f_118', 'f2f_028', 'f2f_013', 'f2f_011', 'f2f_115',
                        'f2f_108', 'f2f_037', 'f2f_034', 'f2f_030', 'f2f_103', 'f2f_111',
                        'f2f_018', 'f2f_026', 'f2f_029', 'f2f_112', 'f2f_106', 'f2f_114')  # N=18
subjects_sntc30_low = ('f2f_015', 'f2f_022', 'f2f_107', 'f2f_021', 'f2f_116', 'f2f_010', 
                       'f2f_104', 'f2f_014', 'f2f_009', 'f2f_024', 'f2f_025', 'f2f_117',
                       'f2f_036', 'f2f_119', 'f2f_012', 'f2f_027')  # N=16
# High: 102, 118, 028, 013, 011, 115, 108, 037, 034, 030, 103, 111, 018, 029, 033,
#     112, 106, 114 
# Low: 015, 022, 107, 117, 021, 116, 010, 104, 014, 009, 024, 025, 036, 119, 041,
#     012, 027

In [3]:
# New groupings for ALL averaging (2020-04-21) #
partial_subjects = ('f2f_009', 'f2f_010', 'f2f_012', 'f2f_013',
                    'f2f_018', 'f2f_021', 'f2f_025', 'f2f_027',
                    'f2f_028', 'f2f_029', 'f2f_030', 'f2f_036',
                    'f2f_037', 'f2f_103', 'f2f_104', 'f2f_108',
                    'f2f_111', 'f2f_114', 'f2f_116', 'f2f_117', 'f2f_119')   # N=21

test_subjects = ['f2f_009', 'f2f_030']

In [4]:
# The actual set to use for the ROI analysis below #
use_subjects = partial_subjects   # test_subjects

In [5]:
# Define PSD parameters and output paths #
yml_file = './f2f_params.yml'
params = mnefun.read_params(yml_file)

params.work_dir = os.path.expanduser('~/Data/MEG/f2f')
params.subjects_dir = os.path.expanduser('~/Data/MEG/anatomy')
params.structurals = params.subjects
params.score = None              # this won't be used
params.subject_indices = None    # this won't be used either
params.subjects = all_subjects   # also

In [6]:
# Tailor parameters for surface or volumetric source space #
SOURCE_TYPE = 'surf';    # options are 'surf' or 'vol'

if SOURCE_TYPE == 'surf':
    params.src = 'oct6'
    params.forward_dir = 'forward_surf'
    params.inverse_dir = 'inverse_surf'
elif SOURCE_TYPE == 'vol':
    params.src = 'vol5'
else:
    raise ValueError('Source type must be "surf" or "vol".')

In [7]:
# Define the band-pass characteristics - these should match the analysis script below #
BANDS_TYPE = 'reduced'

if BANDS_TYPE == 'infant':
    band_names = ('low theta', 'high theta', 'alpha', 'beta', 'gamma')
    band_psddir = 'psd2_fif'
    pattern_suffix = '_infant_srfpsd'
elif BANDS_TYPE == 'reduced':  # new from 2021 Apr
    band_names = ('theta', 'alpha', 'beta', 'gamma')
    band_psddir = 'psd3_fif'
    pattern_suffix = '_reduced_srfpsd'
elif BANDS_TYPE == 'nogamma':  # new from 2021 Apr
    band_names = ('theta', 'alpha', 'beta')
    band_psddir = 'psd4_fif'
    pattern_suffix = '_nogamma_srfpsd'
else:
    band_names = ('theta', 'alpha', 'beta', 'gamma', 'high gamma')
    band_psddir = 'psd1_fif'
    pattern_suffix = '_original_srfpsd'
n_bands = len(band_names)

In [8]:
# Set up spectral estimation parameters #
params.unfreeze()
params.psd_dir = band_psddir
params.psd_params = dict(fmin=0, fmax=60, n_fft=512)
params.freeze()

## Generate PSD STCs
These will go to individual 'psd_fif' directories, along side the sensor-level ones.  The current code below reflects October 2020 changes to create_srcpsds() allowing for either volumetric or surface source spaces. Global normalization is no longer being performed in this notebook.  
Make sure **SOURCE_TYPE** and **BANDS_TYPE** above are set correctly.

In [8]:
# Rerun this cell if python script is updated #
importlib.reload(f2f_analysis_source);

In [23]:
# Check that the loaded analysis script agrees with the BANDS_TYPE setting #
bands_def = f2f_analysis_source.bands_def

assert BANDS_TYPE == f2f_analysis_source.bands_type
assert n_bands == f2f_analysis_source.n_bands
assert all([name in bands_def for name in band_names])

print('The bands to be processed are:')
print(f2f_analysis_source.bands_def)
print('\nThe PSD subdirectory is:', params.psd_dir)

The bands to be processed are:
{'theta': (4.0, 8.0), 'alpha': (8.0, 12.0), 'beta': (12.0, 28.0), 'gamma': (28.0, 60.0)}

The PSD subdirectory is: psd3_fif


In [15]:
# Make source PSDs, stored as regular AND morphed STCs with source-specific equalization #
# The function here has been updated, to differentiate b/w global and local (source-specific) normalization. It also
# removes the "diff" version of source estimation, as that can be derived from the straight talk + ignore STCs.
norm_tag = ''   # to reflect scaling or anything else
create_srcpsds(test_subjects, params, epoch_range=(0,209), morph="14mo_surr",
               activate_proj=True, scaling='local_logmean', extra_tag=norm_tag)

Creating power spectra using reduced frequency bands.
The frequency bands will be:
** {'theta': (4.0, 8.0), 'alpha': (8.0, 12.0), 'beta': (12.0, 28.0), 'gamma': (28.0, 60.0)} **
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
Reading /home/sbierer/Data/MEG/f2f/f2f_009/epochs/All_1sec_80-sss_f2f_009-epo.fif ...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 306)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 306)  idle
    Found the data of interest:
        t =       0.00 ...     997.11 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
746 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
SSP projectors applied...
    ** Adjusting subject label for source space **
Considering frequencies 0 ... 60 Hz
Preparing the inverse operator 

  0%|          |  : 0/210 [00:00<?,       ?it/s]

Considering frequencies 0 ... 60 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 2)
    Created the whitener using a noise covariance matrix with rank 46 (260 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 306 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Reducing data rank 306 -> 46
Using 1 tapers with bandwidth 2.0 Hz on 210 epochs


  0%|          |  : 0/210 [00:00<?,       ?it/s]

  Created psd directory for f2f_009.
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Reading /home/sbierer/Data/MEG/f2f/f2f_010/epochs/All_1sec_80-sss_f2f_010-epo.fif ...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Found the data of interest:
        t =       0.00 ...     997.11 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
645 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
SSP projectors applied...
Considering frequencies 0 ... 60 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Cr

...

  Created psd directory for f2f_118.
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Reading /home/sbierer/Data/MEG/f2f/f2f_119/epochs/All_1sec_80-sss_f2f_119-epo.fif ...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 306)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 306)  idle
    Found the data of interest:
        t =       0.00 ...     997.11 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
713 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
SSP projectors applied...
    ** Adjusting subject label for source space **
Considering frequencies 0 ... 60 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to n

  0%|          |  : 0/210 [00:00<?,       ?it/s]

Considering frequencies 0 ... 60 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 2)
    Created the whitener using a noise covariance matrix with rank 46 (260 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 306 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Reducing data rank 306 -> 46
Using 1 tapers with bandwidth 2.0 Hz on 210 epochs


  0%|          |  : 0/210 [00:00<?,       ?it/s]

  Created psd directory for f2f_119.
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Writing STC to disk...
[done]
Processed source spectra for 35 subjects.


## Create individual and group reports  
The current code below reflects October 2020 changes to create_srcpsd_summary() allowing for either volumetric or surface source spaces and plots.  
As of 2021-02-05, subject reports were only being made for infants. The reports shouldn't overwrite as long as the 'psd1_dir' vs 'psd2_dir' convention is being followed.  
As of 2021-04-15, I switched back to adult bands but with the gammas combined. The directory name is 'psd3_dir'.
Also at this time, condition, diff, and t-diff lims can be passed as a dictionary argument in order to FIX scaling for all subjects or groups. This supercedes the static variables in the \_source.py script, which had to be constantly updated.

In [31]:
# Set up the scaling variables (change defaults only for plots that should have fixed limits) #
INDIV_COND_LIMS, INDIV_DIFF_LIMS, INDIV_TDIFF_LIMS = None, None, None
GROUP_COND_LIMS, GROUP_DIFF_LIMS, GROUP_TDIFF_LIMS = None, None, None

# 1) Old F2F
# GROUP_COND_LIMS = [(12, 15, 20), (12, 15, 20), (12, 15, 20),
#                         (12, 15, 20), (12, 15, 20)]
# GROUP_DIFF_LIMS = [(.35, .40, .45), (.25, .30, .35), (.25, .30, .35),
#                         (.15, .20, .35), (.15, .20, .35)]
# GROUP_TDIFF_LIMS = [(.35, .40, .45), (.20, .25, .30), (.20, .25, .30),
#                         (.12, .15, .18), (.12, .15, .18)]
# 2) Old MMF
# GROUP_DIFF_LIMS = [(1.0, 1.3, 1.5), (0.9, 1.2, 1.3), (0.9, 1.2, 1.3),
#                         (0.9, 1.2, 1.3), (1.8, 2.0, 2.2)]
# GROUP_TDIFF_LIMS = [(3.0, 4.5, 6.0), (3.0, 4.5, 6.0), (3.0, 4.5, 6.0),
#                         (3.0, 4.5, 6.0), (3.0, 4.5, 6.0)]
# 3) Reduced adult bands (2021.04.21)
INDIV_COND_LIMS = None   # defaults to pct with "max" at 99.8% level across conditions!
INDIV_DIFF_LIMS = None
INDIV_TDIFF_LIMS = None

In [32]:
indiv_lims = [INDIV_COND_LIMS, INDIV_DIFF_LIMS, INDIV_TDIFF_LIMS]
for lims_tuple in indiv_lims:
    assert (lims_tuple is None) or (len(lims_tuple) == n_bands)

group_lims = [GROUP_COND_LIMS, GROUP_DIFF_LIMS, GROUP_TDIFF_LIMS]
for lims_tuple in group_lims:
    assert (lims_tuple is None) or (len(lims_tuple) == n_bands)

In [33]:
# Rerun this cell if python script is updated #
importlib.reload(f2f_analysis_source);
from f2f_analysis_source import create_srcpsd_summary

In [34]:
# All subjects individually #
report_path = os.path.expanduser('~/Projects/ILABS/Analyses/F2F/results/srcpsd_2021Apr')

create_srcpsd_summary(partial_subjects, params, mode='individual', model_type = 'surface',
    pattern_in='', pattern_out=pattern_suffix, limits_bytype = indiv_lims,
    path_out=report_path, title='%s Source PSDs');

Compiling report for f2f_009.


AttributeError: 'NoneType' object has no attribute 'GetCurrentCursor'

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_2021Apr/f2f_009_reduced_srfpsd.html
Rendering : Table of Contents
Condition: talk (sbj)
 ... talk (sbj) | THETA
 ... talk (sbj) | ALPHA
 ... talk (sbj) | BETA
 ... talk (sbj) | GAMMA
Condition: ignore (sbj)
 ... ignore (sbj) | THETA
 ... ignore (sbj) | ALPHA
 ... ignore (sbj) | BETA
 ... ignore (sbj) | GAMMA
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA
Compiling report for f2f_010.
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_2021Apr/f2f_010_reduced_srfpsd.html
Rendering : Table of Contents
Condition: talk (sbj)
 ...

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_2021Apr/f2f_037_reduced_srfpsd.html
Rendering : Table of Contents
Condition: talk (sbj)
 ... talk (sbj) | THETA
 ... talk (sbj) | ALPHA
 ... talk (sbj) | BETA
 ... talk (sbj) | GAMMA
Condition: ignore (sbj)
 ... ignore (sbj) | THETA
 ... ignore (sbj) | ALPHA
 ... ignore (sbj) | BETA
 ... ignore (sbj) | GAMMA
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA
Compiling report for f2f_103.
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_2021Apr/f2f_103_reduced_srfpsd.html
Rendering : Table of Contents
Condition: talk (sbj)
 ...

In [11]:
# All subjects average, with morph (as all group reports are) #
report_path = '/home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_XXXXXX'

create_srcpsd_summary(all_subjects, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_ALL' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='Average for ALL Subjects (source eq.)');

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_ALL_original_srfpsd.html
Rendering : Table of Contents
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA
 ... talk-ignore | HIGH GAMMA
Condition: t-test
 ... t-test | THETA
 ... t-test | ALPHA
 ... t-test | BETA
 ... t-test | GAMMA
 ... t-test | HIGH GAMMA
Condition: sum
 ... sum | THETA
 ... sum | ALPHA
 ... sum | BETA
 ... sum | GAMMA
 ... sum | HIGH GAMMA


In [12]:
# SES group averages #
create_srcpsd_summary(subjects_SES_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SEShigh' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='Average for SES High (source eq.)')

create_srcpsd_summary(subjects_SES_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SESlow' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='Average for SES Low (source eq.)')

/home/sbierer/Projects/ILABS/Analyses/F2F/f2f_analysis_source.py:512: RuntimeWarning: All data were zero
  clim=clims[i], backend='mayavi', verbose=False)


Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_SEShigh_original_srfpsd.html
Rendering : Table of Contents
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA
 ... talk-ignore | HIGH GAMMA
Condition: t-test
 ... t-test | THETA
 ... t-test | ALPHA
 ... t-test | BETA
 ... t-test | GAMMA
 ... t-test | HIGH GAMMA
Condition: sum
 ... sum | THETA
 ... sum | ALPHA
 ... sum | BETA
 ... sum | GAMMA
 ... sum | HIGH GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_SESlow_original_srfpsd.html
Rendering : Table of Contents
Condition: talk-i

In [13]:
# 24-month Vocabulary group averages #
create_srcpsd_summary(subjects_vocab24_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_VOCABhigh' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='24-mo Vocabulary High (source eq.)')

create_srcpsd_summary(subjects_vocab24_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_VOCABlow' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='24-mo Vocabulary Low (source eq.)')

/home/sbierer/Projects/ILABS/Analyses/F2F/f2f_analysis_source.py:512: RuntimeWarning: All data were zero
  clim=clims[i], backend='mayavi', verbose=False)


Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_VOCABhigh_original_srfpsd.html
Rendering : Table of Contents
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA
 ... talk-ignore | HIGH GAMMA
Condition: t-test
 ... t-test | THETA
 ... t-test | ALPHA
 ... t-test | BETA
 ... t-test | GAMMA
 ... t-test | HIGH GAMMA
Condition: sum
 ... sum | THETA
 ... sum | ALPHA
 ... sum | BETA
 ... sum | GAMMA
 ... sum | HIGH GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_VOCABlow_original_srfpsd.html
Rendering : Table of Contents
Condition: ta

In [14]:
# 24-month Sentences group averages #
create_srcpsd_summary(subjects_sntc24_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC24high' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='24-mo Sentences High (source eq.)')

create_srcpsd_summary(subjects_sntc24_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC24low' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='24-mo Sentences Low (source eq.)')

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_SNTC24high_original_srfpsd.html
Rendering : Table of Contents
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA
 ... talk-ignore | HIGH GAMMA
Condition: t-test
 ... t-test | THETA
 ... t-test | ALPHA
 ... t-test | BETA
 ... t-test | GAMMA
 ... t-test | HIGH GAMMA
Condition: sum
 ... sum | THETA
 ... sum | ALPHA
 ... sum | BETA
 ... sum | GAMMA
 ... sum | HIGH GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_SNTC24low_original_srfpsd.html
Rendering : Table of Contents
Condition: 

In [15]:
# 30-month Sentences group averages #
create_srcpsd_summary(subjects_sntc30_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC30high' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='30-mo Sentences High (source eq.)')

create_srcpsd_summary(subjects_sntc30_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC30low' + pattern_suffix, limits_bytype = group_lims,
    path_out=report_path, title='30-mo Sentences Low (source eq.)')

/home/sbierer/Projects/ILABS/Analyses/F2F/f2f_analysis_source.py:512: RuntimeWarning: All data were zero
  clim=clims[i], backend='mayavi', verbose=False)


Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_SNTC30high_original_srfpsd.html
Rendering : Table of Contents
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA
 ... talk-ignore | HIGH GAMMA
Condition: t-test
 ... t-test | THETA
 ... t-test | ALPHA
 ... t-test | BETA
 ... t-test | GAMMA
 ... t-test | HIGH GAMMA
Condition: sum
 ... sum | THETA
 ... sum | ALPHA
 ... sum | BETA
 ... sum | GAMMA
 ... sum | HIGH GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Jan/group_SNTC30low_original_srfpsd.html
Rendering : Table of Contents
Condition: 

## Group reports for talk-only and ignore-only maps
2020-02-06 update using the _grpplot_mode_ argument.

In [17]:
# Rerun this cell if python script is updated #
importlib.reload(f2f_analysis_source);
from f2f_analysis_source import create_srcpsd_summary

In [36]:
# All subjects average, with morph (as all group reports are) #
report_path = os.path.expanduser('~/Projects/ILABS/Analyses/F2F/results/srcpsd_2021Apr')
title_str = f'Average for ALL Subjects (N={len(partial_subjects)})'

create_srcpsd_summary(partial_subjects, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_ALL_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title=title_str);

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_2021Apr/group_ALL_COND_reduced_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | THETA
 ... talk | ALPHA
 ... talk | BETA
 ... talk | GAMMA
Condition: ignore
 ... ignore | THETA
 ... ignore | ALPHA
 ... ignore | BETA
 ... ignore | GAMMA
Condition: talk-ignore
 ... talk-ignore | THETA
 ... talk-ignore | ALPHA
 ... talk-ignore | BETA
 ... talk-ignore | GAMMA


In [46]:
# SES group averages #
create_srcpsd_summary(subjects_SES_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SEShigh_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='Average for SES High (source eq.)')

create_srcpsd_summary(subjects_SES_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SESlow_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='Average for SES Low (source eq.)')

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_SEShigh_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | LOW THETA
 ... talk | HIGH THETA
 ... talk | ALPHA
 ... talk | BETA
 ... talk | GAMMA
Condition: ignore
 ... ignore | LOW THETA
 ... ignore | HIGH THETA
 ... ignore | ALPHA
 ... ignore | BETA
 ... ignore | GAMMA
Condition: mean
 ... mean | LOW THETA
 ... mean | HIGH THETA
 ... mean | ALPHA
 ... mean | BETA
 ... mean | GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_SESlow_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | LOW THE

In [47]:
# 24-month Vocabulary group averages #
create_srcpsd_summary(subjects_vocab24_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_VOCABhigh_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='24-mo Vocabulary High (source eq.)')

create_srcpsd_summary(subjects_vocab24_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_VOCABlow_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='24-mo Vocabulary Low (source eq.)')

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_VOCABhigh_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | LOW THETA
 ... talk | HIGH THETA
 ... talk | ALPHA
 ... talk | BETA
 ... talk | GAMMA
Condition: ignore
 ... ignore | LOW THETA
 ... ignore | HIGH THETA
 ... ignore | ALPHA
 ... ignore | BETA
 ... ignore | GAMMA
Condition: mean
 ... mean | LOW THETA
 ... mean | HIGH THETA
 ... mean | ALPHA
 ... mean | BETA
 ... mean | GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_VOCABlow_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | LOW

In [48]:
# 24-month Sentences group averages #
create_srcpsd_summary(subjects_sntc24_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC24high_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='24-mo Sentences High (source eq.)')

create_srcpsd_summary(subjects_sntc24_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC24low_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='24-mo Sentences Low (source eq.)')

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_SNTC24high_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | LOW THETA
 ... talk | HIGH THETA
 ... talk | ALPHA
 ... talk | BETA
 ... talk | GAMMA
Condition: ignore
 ... ignore | LOW THETA
 ... ignore | HIGH THETA
 ... ignore | ALPHA
 ... ignore | BETA
 ... ignore | GAMMA
Condition: mean
 ... mean | LOW THETA
 ... mean | HIGH THETA
 ... mean | ALPHA
 ... mean | BETA
 ... mean | GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_SNTC24low_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | L

In [49]:
# 30-month Sentences group averages #
create_srcpsd_summary(subjects_sntc30_high, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC30high_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='30-mo Sentences High (source eq.)')

create_srcpsd_summary(subjects_sntc30_low, params, mode='group', model_type = 'surface',
    pattern_in='', pattern_out='_SNTC30low_COND' + pattern_suffix, grpplot_mode = 'condition',
    limits_bytype = group_lims, path_out=report_path, title='30-mo Sentences Low (source eq.)')

Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_SNTC30high_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | LOW THETA
 ... talk | HIGH THETA
 ... talk | ALPHA
 ... talk | BETA
 ... talk | GAMMA
Condition: ignore
 ... ignore | LOW THETA
 ... ignore | HIGH THETA
 ... ignore | ALPHA
 ... ignore | BETA
 ... ignore | GAMMA
Condition: mean
 ... mean | LOW THETA
 ... mean | HIGH THETA
 ... mean | ALPHA
 ... mean | BETA
 ... mean | GAMMA
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css
Saving report to location /home/sbierer/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2021Feb/group_SNTC30low_COND_infant_srfpsd.html
Rendering : Table of Contents
Condition: talk
 ... talk | L

## Extract label PSDs for ROI analysis - surface source space
Folder targets:
- srcpsd_groups_2021Jan
- srcpsd_2021Apr
- srcpsd_groups_2021Dec
- srcpsd_groups_2022Jan  (added transversetemporal, bankssts)

In [9]:
# Pick flavor of PSD STCs to load #
stc_pattern = ''   # without a "_"

In [10]:
# Check AGAIN that the loaded analysis script agrees with the BANDS_TYPE setting #
bands_def = f2f_analysis_source.bands_def

assert BANDS_TYPE == f2f_analysis_source.bands_type
assert n_bands == f2f_analysis_source.n_bands
assert all([name in bands_def for name in band_names])

print('The bands to be processed are:')
print(f2f_analysis_source.bands_def)
print('\nThe PSD subdirectory is:', params.psd_dir)

The bands to be processed are:
{'theta': (4.0, 8.0), 'alpha': (8.0, 12.0), 'beta': (12.0, 28.0), 'gamma': (28.0, 60.0)}

The PSD subdirectory is: psd3_fif


In [11]:
# Define the labels and label set #
use_parc = 'aparc'
source_subject = '14mo_surr'

labels_of_interest = ['posteriorcingulate', 'medialorbitofrontal', 'lateralorbitofrontal',
                      'insula', 'parsopercularis', 'parsorbitalis', 'parstriangularis',
                      'middletemporal', 'inferiortemporal', 'superiortemporal', 'temporalpole',
                      'inferiorparietal', 'superiorparietal', 'rostralanteriorcingulate',
                      'transversetemporal', 'bankssts', 'caudalanteriorcingulate']

# Define csv path #
roi_path = os.path.expanduser('~/Projects/ILABS/Analyses/F2F/results/srcpsd_groups_2022Jan')

In [12]:
# Read in the label set #
parc_exists = os.path.isfile(os.path.join(params.subjects_dir, source_subject, 'label', f'lh.{use_parc}.annot'))
print(parc_exists)  # check for the existence of the chosen Freesurfer parcellation set

labels = mne.read_labels_from_annot(source_subject, use_parc, subjects_dir=params.subjects_dir)
for ll in labels:
    print(ll.name)

True
Reading labels from parcellation...
   read 34 labels from /home/sbierer/Data/MEG/anatomy/14mo_surr/label/lh.aparc.annot
   read 34 labels from /home/sbierer/Data/MEG/anatomy/14mo_surr/label/rh.aparc.annot
bankssts-lh
bankssts-rh
caudalanteriorcingulate-lh
caudalanteriorcingulate-rh
caudalmiddlefrontal-lh
caudalmiddlefrontal-rh
cuneus-lh
cuneus-rh
entorhinal-lh
entorhinal-rh
frontalpole-lh
frontalpole-rh
fusiform-lh
fusiform-rh
inferiorparietal-lh
inferiorparietal-rh
inferiortemporal-lh
inferiortemporal-rh
insula-lh
insula-rh
isthmuscingulate-lh
isthmuscingulate-rh
lateraloccipital-lh
lateraloccipital-rh
lateralorbitofrontal-lh
lateralorbitofrontal-rh
lingual-lh
lingual-rh
medialorbitofrontal-lh
medialorbitofrontal-rh
middletemporal-lh
middletemporal-rh
paracentral-lh
paracentral-rh
parahippocampal-lh
parahippocampal-rh
parsopercularis-lh
parsopercularis-rh
parsorbitalis-lh
parsorbitalis-rh
parstriangularis-lh
parstriangularis-rh
pericalcarine-lh
pericalcarine-rh
postcentral-lh
po

In [13]:
# Define plotting SURFACE source space (14-month surrogate) #
src_name = os.path.join(params.subjects_dir, '14mo_surr', 'bem', '14mo_surr-oct-6-src.fif')
src = mne.read_source_spaces(src_name)

print(src[0]['rr'].shape, src[1]['rr'].shape)  # same shapes as for any subject's inv['src']

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
(99704, 3) (100758, 3)


In [14]:
# Build up the column names and define TWO empty pandas DataFrame for each band #
# One for regular dB units, the other for expressing as %.
columns = []
for label in labels_of_interest:
    for pattern in ('-lh-talk', '-lh-ignore','-rh-talk', '-rh-ignore'):
        columns.append(label + pattern)

Table_list, TableLinear_list = [], []
for bb in range(n_bands):
    df = pd.DataFrame(data=None, columns=columns, index = use_subjects)
    Table_list.append(df)
    TableLinear_list.append(df.copy())

In [15]:
# Execute the loop to fill the data frames #
labels_list = [ll.name for ll in labels]

for subject in use_subjects:
    print(f'Processing subject {subject}...   ')
    
    stc_file = os.path.join(params.work_dir, subject, params.psd_dir,
                            subject + stc_pattern + '_talk_psd_morphed-lh.stc')
    stc_talk = mne.read_source_estimate(stc_file)
    stc_file = os.path.join(params.work_dir, subject, params.psd_dir,
                            subject + stc_pattern + '_ignore_psd_morphed-lh.stc')
    stc_ignore = mne.read_source_estimate(stc_file)

    for label in labels_of_interest:
        for hemi in ('-lh', '-rh'):
            entry = label + hemi
            idx = labels_list.index(entry)
            
            label_talk = mne.extract_label_time_course(stc_talk, labels[idx], src,
                                                       mode='mean', verbose=False)
            label_ignore = mne.extract_label_time_course(stc_ignore, labels[idx], src,
                                                         mode='mean', verbose=False)
            for bb in range(n_bands):
                col = label + hemi + '-talk'
                Table_list[bb].loc[subject, col] = label_talk[0, bb]
                TableLinear_list[bb].loc[subject, col] = 10**(label_talk[0, bb]/10 + 2)
                
                col = label + hemi + '-ignore'
                Table_list[bb].loc[subject, col] = label_ignore[0, bb]
                TableLinear_list[bb].loc[subject, col] = 10**(label_ignore[0, bb]/10 + 2)

Processing subject f2f_009...   
Processing subject f2f_010...   
Processing subject f2f_012...   
Processing subject f2f_013...   
Processing subject f2f_018...   
Processing subject f2f_021...   
Processing subject f2f_025...   
Processing subject f2f_027...   
Processing subject f2f_028...   
Processing subject f2f_029...   
Processing subject f2f_030...   
Processing subject f2f_036...   
Processing subject f2f_037...   
Processing subject f2f_103...   
Processing subject f2f_104...   
Processing subject f2f_108...   
Processing subject f2f_111...   
Processing subject f2f_114...   
Processing subject f2f_116...   
Processing subject f2f_117...   
Processing subject f2f_119...   


In [16]:
# Look at one of the dB tables #
Table_list[0].head()

,posteriorcingulate-lh-talk,posteriorcingulate-lh-ignore,posteriorcingulate-rh-talk,posteriorcingulate-rh-ignore,medialorbitofrontal-lh-talk,medialorbitofrontal-lh-ignore,medialorbitofrontal-rh-talk,medialorbitofrontal-rh-ignore,lateralorbitofrontal-lh-talk,lateralorbitofrontal-lh-ignore,...,transversetemporal-rh-talk,transversetemporal-rh-ignore,bankssts-lh-talk,bankssts-lh-ignore,bankssts-rh-talk,bankssts-rh-ignore,caudalanteriorcingulate-lh-talk,caudalanteriorcingulate-lh-ignore,caudalanteriorcingulate-rh-talk,caudalanteriorcingulate-rh-ignore
f2f_009,-2.57166,-3.0623,-2.62738,-2.98994,-3.14098,-3.69546,-2.87354,-3.55727,-3.44622,-3.94553,...,-2.34655,-2.98715,-3.05303,-4.05274,-2.42217,-3.26992,-2.70847,-3.59617,-2.69259,-3.32685
f2f_010,-5.09009,-3.86754,-4.86828,-4.18862,-5.19564,-4.48024,-3.96483,-3.74745,-5.39708,-4.75751,...,-4.02023,-3.24355,-6.43874,-6.22188,-4.53155,-3.70642,-6.12943,-4.56991,-4.99484,-4.51285
f2f_012,-2.21368,-2.43412,-2.42355,-2.24301,-2.64914,-3.25262,-2.55975,-3.1467,-3.12534,-3.36432,...,-2.47315,-2.8599,-3.56023,-3.65951,-2.87947,-2.40734,-2.49662,-2.81532,-2.39683,-2.33965
f2f_013,-2.88513,-3.48185,-2.84469,-4.35624,-3.4079,-2.85871,-3.11194,-3.23208,-3.70753,-2.48065,...,-2.91601,-5.37015,-4.74592,-3.02109,-3.28617,-6.42443,-3.42978,-3.22754,-3.24397,-3.62332
f2f_018,-2.75142,-4.19372,-2.0984,-4.5539,-1.96046,-3.88683,-2.11035,-4.72913,-1.99729,-4.22099,...,-1.72264,-5.5452,-4.03123,-4.71137,-2.07065,-6.001,-2.29325,-5.1861,-2.05494,-4.64509


In [17]:
# Look at one of the linear tables #
TableLinear_list[0].head()

,posteriorcingulate-lh-talk,posteriorcingulate-lh-ignore,posteriorcingulate-rh-talk,posteriorcingulate-rh-ignore,medialorbitofrontal-lh-talk,medialorbitofrontal-lh-ignore,medialorbitofrontal-rh-talk,medialorbitofrontal-rh-ignore,lateralorbitofrontal-lh-talk,lateralorbitofrontal-lh-ignore,...,transversetemporal-rh-talk,transversetemporal-rh-ignore,bankssts-lh-talk,bankssts-lh-ignore,bankssts-rh-talk,bankssts-rh-ignore,caudalanteriorcingulate-lh-talk,caudalanteriorcingulate-lh-ignore,caudalanteriorcingulate-rh-talk,caudalanteriorcingulate-rh-ignore
f2f_009,55.3139,49.4048,54.6087,50.235,48.5179,42.7026,51.5996,44.0832,45.2249,40.3132,...,58.2566,50.2673,49.5104,39.3302,57.251,47.0986,53.5986,43.6901,53.7949,46.4853
f2f_010,30.9736,41.0437,32.5965,38.1187,30.2298,35.6431,40.1344,42.1944,28.8597,33.4387,...,39.6257,47.3855,22.7053,23.8678,35.2245,42.595,24.3813,34.9147,31.6603,35.3765
f2f_012,60.0664,57.0936,57.2328,59.6621,54.3358,47.2866,55.4658,48.4541,48.6929,46.0859,...,56.5829,51.7618,44.0532,43.0575,51.5292,57.4468,56.2779,52.2959,57.586,58.3492
f2f_013,51.4621,44.8554,51.9435,36.6755,45.6257,51.776,48.8434,47.5108,42.5841,56.4852,...,51.0974,29.0392,33.528,49.8759,46.9227,22.7801,45.3965,47.5605,47.3808,43.4178
f2f_018,53.0711,38.0739,61.6822,35.0437,63.6728,40.8618,61.5127,33.6579,63.1351,37.8356,...,67.2568,27.892,39.5254,33.7959,62.0776,25.1131,58.9759,30.2963,62.3026,34.3155


In [20]:
# Save the results as CSV files #
prefix = 'roi_' + BANDS_TYPE.upper() + '_'
suffix = stc_pattern + '.csv'

for bb in range(n_bands):
    band_str = band_names[bb].replace(' ','').upper()
    file = os.path.join(roi_path, prefix + band_str + suffix)
    Table_list[bb].to_csv(file)

prefix = 'roilinear_' + BANDS_TYPE.upper() + '_'
suffix = stc_pattern + '.csv'

for bb in range(n_bands):
    band_str = band_names[bb].replace(' ','').upper()
    file = os.path.join(roi_path, prefix + band_str + suffix)
    TableLinear_list[bb].to_csv(file)

## Repeat label extraction for all the rest of the labels

In [21]:
# Construct a list of the rest of the labels #
labels_all_the_rest = []
for label in labels:
    if label.name[-2:] == 'rh':  # don't repeat lh and rh
        continue
    if label.name[:-3] not in labels_of_interest:
        labels_all_the_rest.append(label.name[:-3])

print(f'{len(labels_all_the_rest)} remaining labels.')

17 remaining labels.


In [22]:
# Define another empty pandas DataFrame for each band #
columns = []
for label in labels_all_the_rest:
    for pattern in ('-lh-talk', '-lh-ignore','-rh-talk', '-rh-ignore'):
        columns.append(label + pattern)

Table_list = []
for bb in range(n_bands):
    df = pd.DataFrame(data=None, columns=columns, index = all_subjects)
    Table_list.append(df)

In [23]:
# Fill the data frames #
labels_list = [ll.name for ll in labels]

for subject in all_subjects:
    print(f'Processing subject {subject}...   ')
    
    stc_file = os.path.join(params.work_dir, subject, params.psd_dir,
                            subject + stc_pattern + '_talk_psd_morphed-lh.stc')
    stc_talk = mne.read_source_estimate(stc_file)
    stc_file = os.path.join(params.work_dir, subject, params.psd_dir,
                            subject + stc_pattern + '_ignore_psd_morphed-lh.stc')
    stc_ignore = mne.read_source_estimate(stc_file)

    for label in labels_all_the_rest:
        for hemi in ('-lh', '-rh'):
            entry = label + hemi
            idx = labels_list.index(entry)
            
            label_talk = mne.extract_label_time_course(stc_talk, labels[idx], src,
                                                       mode='mean', verbose=False)
            label_ignore = mne.extract_label_time_course(stc_ignore, labels[idx], src,
                                                         mode='mean', verbose=False)
            for bb in range(n_bands):
                col = label + hemi + '-talk'
                Table_list[bb].loc[subject, col] = label_talk[0, bb]
                
                col = label + hemi + '-ignore'
                Table_list[bb].loc[subject, col] = label_ignore[0, bb]

Processing subject f2f_009...   
Processing subject f2f_010...   
Processing subject f2f_011...   
Processing subject f2f_012...   
Processing subject f2f_013...   
Processing subject f2f_014...   
Processing subject f2f_015...   
Processing subject f2f_018...   
Processing subject f2f_021...   
Processing subject f2f_022...   
Processing subject f2f_024...   
Processing subject f2f_025...   
Processing subject f2f_026...   
Processing subject f2f_027...   
Processing subject f2f_028...   
Processing subject f2f_029...   
Processing subject f2f_030...   
Processing subject f2f_034...   
Processing subject f2f_036...   
Processing subject f2f_037...   
Processing subject f2f_038...   
Processing subject f2f_102...   
Processing subject f2f_103...   
Processing subject f2f_104...   
Processing subject f2f_106...   
Processing subject f2f_107...   
Processing subject f2f_108...   
Processing subject f2f_111...   
Processing subject f2f_112...   
Processing subject f2f_114...   
Processing

In [24]:
# Look at one of the tables #
Table_list[1].head()

,caudalmiddlefrontal-lh-talk,caudalmiddlefrontal-lh-ignore,caudalmiddlefrontal-rh-talk,caudalmiddlefrontal-rh-ignore,cuneus-lh-talk,cuneus-lh-ignore,cuneus-rh-talk,cuneus-rh-ignore,entorhinal-lh-talk,entorhinal-lh-ignore,...,rostralmiddlefrontal-rh-talk,rostralmiddlefrontal-rh-ignore,superiorfrontal-lh-talk,superiorfrontal-lh-ignore,superiorfrontal-rh-talk,superiorfrontal-rh-ignore,supramarginal-lh-talk,supramarginal-lh-ignore,supramarginal-rh-talk,supramarginal-rh-ignore
f2f_009,-7.00228,-8.08446,-6.51142,-7.33774,-7.4697,-7.92128,-8.10824,-8.27253,-7.61017,-7.97029,...,-6.70068,-7.21451,-6.79902,-7.70771,-6.76771,-7.4368,-7.60267,-8.21083,-7.84043,-8.43005
f2f_010,-15.3213,-15.678,-10.0007,-9.97859,-12.298,-12.8017,-10.953,-11.1727,-9.7795,-10.1621,...,-10.6189,-10.8169,-12.4215,-12.6038,-11.3514,-11.4373,-15.8553,-16.6298,-9.96985,-9.96462
f2f_011,-11.8343,-10.5593,-8.6766,-7.34764,-11.0556,-9.29075,-11.4592,-9.87323,-9.65351,-8.45825,...,-9.72738,-8.78191,-11.499,-10.2594,-10.0166,-8.64315,-10.07,-8.95456,-10.3807,-9.6253
f2f_012,-13.5817,-13.3624,-9.33092,-9.59486,-12.9343,-11.9323,-10.8207,-10.1398,-9.22066,-9.77396,...,-11.7049,-11.8997,-10.9065,-11.0569,-10.296,-10.4332,-14.919,-14.3293,-9.97495,-9.965
f2f_013,-13.1379,-11.4038,-7.64844,-9.35817,-8.5059,-10.7663,-9.03303,-11.1653,-8.91654,-9.81492,...,-7.86535,-8.69111,-10.0255,-10.3668,-8.84187,-9.56744,-13.2341,-11.7479,-8.09524,-10.5635


In [25]:
# Save the results as CSV files #
prefix = 'roi_extra_' + BANDS_TYPE.upper() + '_'
suffix = '.csv'

for bb in range(n_bands):
    band_str = band_names[bb].replace(' ','').upper()
    file = os.path.join(roi_path, prefix + band_str + suffix)
    Table_list[bb].to_csv(file)